### **Data Preprocessing**



In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf  # Version 1.0.0 (some previous versions are used in past commits)
from sklearn import metrics
import os
import psutil
import time

In [2]:
# Useful Constants

# Those are separate normalised input features for the neural network
INPUT_SIGNAL_TYPES = [
    "x_accel",
    "y_accel",
    "z_accel",
]

# Output classes to learn how to classify
LABELS = [
    "Walking", 
    "Jogging", 
    "Uptairs", 
    "Downstairs", 
    "Sitting", 
    "Standing"
] 

In [3]:
from numpy import genfromtxt
import pandas as pd
x = genfromtxt('../Data/WISDM_x.csv', delimiter=',')
y = pd.read_csv('../Data/WISDM_y.csv')

In [4]:
def create_series(x,y,timestep,overlap):

    slide_step = int(timestep*(1-overlap))
    data_num = int((len(x)/slide_step)-1)
  
    dataset = np.ndarray(shape=(data_num,timestep,len(x[0])))
    labels = list()

    for i in range(data_num):
        labels.append(y[slide_step*(i+1)-1])
        for j in range(timestep):
            dataset[i,j,:] = x[slide_step*i+j,:]

    return dataset,np.array(labels)


In [5]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
le = LabelEncoder()
y = le.fit_transform(y).reshape(-1,1)


ohe = OneHotEncoder()
y = ohe.fit_transform(y).toarray()


/Users/sandeep/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [6]:
# y_pred.append("5")
dataset,labels = create_series(x,y,16,0.5)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(dataset,labels,test_size=0.3, random_state=412)
# X_test, X_val, y_test, y_val = train_test_split(X_toSplit,y_toSplit,test_size=0.25, random_state=412)

In [7]:
# Input Data 

training_data_count = len(X_train)  # 96092 training series (with 50% overlap between each serie)
test_data_count = len(X_test)  # 41183 testing series
n_steps = len(X_train[0])  # 16 timesteps per series
n_input = len(X_train[0][0])  # 3 input parameters per timestep


# LSTM Neural Network's internal structure
# n_hidden = 32 # Hidden layer num of features
n_classes = 6 # Total classes (should go up, or should go down)


# Training 

learning_rate = 0.0025
lambda_loss_amount = 0.0015
training_iters = training_data_count * 300  # Loop 300 times on the dataset
batch_size = 64
display_iter = 30000  # To show test set accuracy during training


# Some debugging info

print("Some useful info to get an insight on dataset's shape and normalisation:")
print("(X shape, y shape, every X's mean, every X's standard deviation)")
print(f'X_train_shape: {X_train.shape}, y_train.shape: {y_train.shape}')
print(f'X_test_shape: {X_test.shape}, y_test.shape: {y_test.shape}')
print("The dataset is therefore properly normalised, as expected, but not yet one-hot encoded.")

Some useful info to get an insight on dataset's shape and normalisation:
(X shape, y shape, every X's mean, every X's standard deviation)
X_train_shape: (96092, 16, 3), y_train.shape: (96092, 6)
X_test_shape: (41183, 16, 3), y_test.shape: (41183, 6)
The dataset is therefore properly normalised, as expected, but not yet one-hot encoded.


### **Model**

In [8]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


model = keras.Sequential()
model.add(layers.LSTM(256, return_sequences=True, input_shape=(n_steps, n_input)))
model.add(layers.Dropout(0.2))
model.add(layers.LSTM(128))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(6, activation='softmax'))

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10)

model.compile(loss="categorical_crossentropy",optimizer='adam',metrics=["accuracy"])

history = model.fit(X_train,y_train,batch_size=64,epochs=20,validation_split=0.1,callbacks=[callback])

Epoch 1/20
1352/1352 [==============================] - 48s 35ms/step - loss: 0.4213 - accuracy: 0.8519 - val_loss: 0.2816 - val_accuracy: 0.9048
Epoch 2/20
1352/1352 [==============================] - 48s 35ms/step - loss: 0.2116 - accuracy: 0.9289 - val_loss: 0.1690 - val_accuracy: 0.9445
Epoch 3/20
1352/1352 [==============================] - 49s 36ms/step - loss: 0.1527 - accuracy: 0.9492 - val_loss: 0.1477 - val_accuracy: 0.9528
Epoch 4/20
1352/1352 [==============================] - 48s 36ms/step - loss: 0.1231 - accuracy: 0.9591 - val_loss: 0.1494 - val_accuracy: 0.9522
Epoch 5/20
1352/1352 [==============================] - 49s 36ms/step - loss: 0.1014 - accuracy: 0.9661 - val_loss: 0.1259 - val_accuracy: 0.9611
Epoch 6/20
1352/1352 [==============================] - 51s 38ms/step - loss: 0.0846 - accuracy: 0.9719 - val_loss: 0.1254 - val_accuracy: 0.9613
Epoch 7/20
1352/1352 [==============================] - 55s 41ms/step - loss: 0.0724 - accuracy: 0.9764 - val_loss: 0.1184 -

KeyboardInterrupt: 

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
model.save('/Users/sandeep/Desktop/BUCourses/Project/saved_models/LSTM')

model.summary()


In [8]:
from helper import compute_metrics_base, compute_metrics, measure_cpu_utilization_and_run


#### Base model

In [11]:
model_dir = '/Users/sandeep/Desktop/BUCourses/Project/saved_models/LSTM'
model = tf.keras.models.load_model(model_dir)
average_cpu_utilization, elapsed_time, result = measure_cpu_utilization_and_run(
                                                compute_metrics_base, model, X_test, y_test, model_dir)

print(f"Elapsed time: {elapsed_time:.2f} seconds")
print(f"Average CPU utilization: {average_cpu_utilization:.2f}%")


1287/1287 [==============================] - 8s 6ms/step
Size of the model: 21649.87 KB
Accuracy on the test set: 96.56%
Elapsed time: 8.04 seconds
Average CPU utilization: 27.20%


### Dynamic Range Quantization

In [12]:
#Implemented Dynamic Range Quantization

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow_model_optimization.sparsity import keras as sparsity

#Load saved model
model = tf.keras.models.load_model('/Users/sandeep/Desktop/BUCourses/Project/saved_models/LSTM')


converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,
    tf.lite.OpsSet.SELECT_TF_OPS
]
converter._experimental_lower_tensor_list_ops = False

# Now, convert the model
dynamicQ_tflite_model = converter.convert()


# Save the quantized model to a file
with open('/Users/sandeep/Desktop/BUCourses/Project/saved_models/dynamicQ_model.tflite', 'wb') as f:
    f.write(dynamicQ_tflite_model)


INFO:tensorflow:Assets written to: /var/folders/l4/2y2z7y6s4f39zqdkw_9jv51h0000gn/T/tmpqwug3ph0/assets


INFO:tensorflow:Assets written to: /var/folders/l4/2y2z7y6s4f39zqdkw_9jv51h0000gn/T/tmpqwug3ph0/assets
2023-10-20 11:49:59.588380: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-10-20 11:49:59.588565: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2023-10-20 11:49:59.589550: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /var/folders/l4/2y2z7y6s4f39zqdkw_9jv51h0000gn/T/tmpqwug3ph0
2023-10-20 11:49:59.595374: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2023-10-20 11:49:59.595380: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /var/folders/l4/2y2z7y6s4f39zqdkw_9jv51h0000gn/T/tmpqwug3ph0
2023-10-20 11:49:59.613210: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:375] MLIR V1 optimization pass is not enabled
2023-10-20 11:49:59.616145: I tensorflow/cc/saved_model/load

In [13]:


average_cpu_utilization, elapsed_time, result = measure_cpu_utilization_and_run(
                                                compute_metrics, dynamicQ_tflite_model, X_test, y_test, input_type='float32'
                                                )

print(f"Elapsed time: {elapsed_time:.2f} seconds")
print(f"Average CPU utilization: {average_cpu_utilization:.2f}%")


INFO: Created TensorFlow Lite delegate for select TF ops.
INFO: TfLiteFlexDelegate delegate: 4 nodes delegated out of 21 nodes with 3 partitions.

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


Size of TFLite model: 482.24 KB
Accuracy on the test set: 96.55%
Elapsed time: 34.47 seconds
Average CPU utilization: 21.35%
